# **Transformación de tabla physical_specs**
##### **Ejemplo de una carga incremental**

##### **01.- Leer tabla con spark.sql**

In [1]:
%run common_functions

StatementMeta(, e7b183ef-1c08-4808-a55e-569c8eae8f0f, 4, Finished, Available, Finished)

In [2]:
df1 = spark.sql("SELECT * FROM lh_gold.dbo.fact_device")

StatementMeta(, e7b183ef-1c08-4808-a55e-569c8eae8f0f, 5, Finished, Available, Finished)

##### **02.- Eliminar todas las columnas menos una**

In [7]:
df2 = df1.select("released_announced")

StatementMeta(, e7b183ef-1c08-4808-a55e-569c8eae8f0f, 10, Finished, Available, Finished)

##### **03.- Eliminar duplicados**

In [9]:
df3 = df2.dropDuplicates()

StatementMeta(, e7b183ef-1c08-4808-a55e-569c8eae8f0f, 12, Finished, Available, Finished)

##### **04.- Cambiar nombre de columna**

In [11]:
df4 = df3.withColumnRenamed("released_announced", "date")


StatementMeta(, e7b183ef-1c08-4808-a55e-569c8eae8f0f, 14, Finished, Available, Finished)

##### **05.- Ordenar de manera ascendente**

In [16]:
df5 = df4.orderBy("date")  # por defecto es ascendente

StatementMeta(, e7b183ef-1c08-4808-a55e-569c8eae8f0f, 19, Finished, Available, Finished)

##### **06.- Agregar columnas de time intelligence a time**

In [18]:
from pyspark.sql.functions import year, month, dayofmonth, date_format, quarter, concat, lit
time_final_df = df5.withColumn("year", year("date")) \
                            .withColumn("month", month("date")) \
                            .withColumn("day", dayofmonth("date")) \
                            .withColumn("month_name", date_format("date", "MMMM")) \
                            .withColumn("day_name", date_format("date", "EEEE")) \
                            .withColumn("year", year("date")) \
                            .withColumn("quarter", concat(lit("Q"), quarter("date")))

StatementMeta(, e7b183ef-1c08-4808-a55e-569c8eae8f0f, 21, Finished, Available, Finished)

In [19]:
display(time_final_df.head(10))

StatementMeta(, e7b183ef-1c08-4808-a55e-569c8eae8f0f, 22, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0ad5e67f-ef7b-40ab-86c8-0d5e6eeae840)

##### **07.- Escribir datos en lh_gold**

In [20]:
time_final_df.write.format("delta").mode("overwrite").saveAsTable("dim_time")

StatementMeta(, e7b183ef-1c08-4808-a55e-569c8eae8f0f, 23, Finished, Available, Finished)

In [21]:
%%sql
select * from dim_time

StatementMeta(, e7b183ef-1c08-4808-a55e-569c8eae8f0f, 24, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 7 fields>